# Dataset

In [3]:
run ../utils/preprocess.py

In [6]:
category = "Video_Games"

C.INPUT_DATA_PATH = '../../data/input'
C.JSON_DATA_PATH = '../../data/json_data'
C.QA_DATA_PATH = '../../data/answers_multiple'
C.REVIEWS_DATA_PATH = '../../data/reviews_small'

#Generate Raw, Filter, Train/Val/Test Datasets

In [7]:
generate_raw_data(category)
generate_split_data(category)

In [11]:
def get_counts(df):
    questionId = 0
    answerId = 0
    for index, row in df.iterrows():
        questionsList = row[C.QUESTIONS_LIST]
        for question in questionsList:
            if C.TEXT in question:
                text = question[C.TEXT]
                questionId += 1

                for answer in question[C.ANSWERS]:
                    text = answer[C.TEXT]
                    answerId += 1
    return questionId, answerId

In [8]:
df_raw = get_raw_dataframe(category)

In [9]:
df_raw

,asin,questionsList,reviewsList
0,B0000512IE,[{'text': 'Can I use these with Window 8/8.1?'...,[{'text': 'I use this product with Microsoft F...
1,B00006B84X,[{'text': 'CH Fighter Stick vs Saitek X52 Havi...,[{'text': 'Bought this together with the rudde...
2,B00009W5WB,[{'text': 'Is this the cable for older style ...,[{'text': 'Much longer than the picture would ...
3,B00030AX3Q,"[{'text': 'does this work with windows 8.1', '...",[{'text': 'I got this controller to play 2D ga...
4,B000BTBMWA,[{'text': 'Can you fit an HDMI cable under it?...,[{'text': 'Like pretty much all Monster produc...


#Number of questionAnswer Pairs in RawDataset

In [12]:
get_counts(df_raw)

(30, 104)

#Number of questionAnswer Pairs in TrainDataset

In [13]:
df_train = get_split_dataframe(category, 'train')

In [14]:
get_counts(df_train)

(22, 83)